In [1]:
!pip uninstall torch torchvision torchaudio detectron2 -y
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Found existing installation: torch 2.6.0+cpu
Uninstalling torch-2.6.0+cpu:
  Successfully uninstalled torch-2.6.0+cpu
Found existing installation: torchvision 0.21.0+cpu
Uninstalling torchvision-0.21.0+cpu:
  Successfully uninstalled torchvision-0.21.0+cpu
Found existing installation: torchaudio 2.6.0+cpu
Uninstalling torchaudio-2.6.0+cpu:
  Successfully uninstalled torchaudio-2.6.0+cpu
Found existing installation: detectron2 0.6
Uninstalling detectron2-0.6:
  Successfully uninstalled detectron2-0.6
Looking in indexes: https://download.pytorch.org/whl/cpu
  Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (26 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchvision-0.21.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cpu/torchaudio-2.6.0%2Bcpu-cp311-cp311-linux_x86_64.whl.metadata (6.6 kB)
Using cached https://download.pytorch.org/whl/cpu/torch-2.6.0%2Bcpu-cp311-cp

In [2]:
!pip uninstall detectron2 -y
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
  Preparing metadata (setup.py) ... done
  Running setup.py develop for detectron2


In [3]:
import torch, torchvision


import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()


import numpy as np
import cv2
from google.colab.patches import cv2_imshow


from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
import detectron2
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file, get_checkpoint_url
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer
import cv2
from google.colab.patches import cv2_imshow
import random
from detectron2.data import DatasetCatalog, MetadataCatalog


(f'Exist?: {os.path.exists("/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json")}')

register_coco_instances("my_train", {}, "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json", "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images")
register_coco_instances("my_val", {}, "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json", "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images")


def create_small_dataset(dataset_name, sample_size=5):
    dataset = DatasetCatalog.get(dataset_name)
    small_dataset = random.sample(dataset, sample_size)
    DatasetCatalog.register("my_train_small", lambda: small_dataset)
    MetadataCatalog.get("my_train_small").set(thing_classes=MetadataCatalog.get(dataset_name).thing_classes)

create_small_dataset("my_train", sample_size=5)


cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_train_small",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 2  
cfg.SOLVER.BASE_LR = 0.00025  
cfg.SOLVER.MAX_ITER = 500  
cfg.SOLVER.STEPS = []  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.OUTPUT_DIR = "/gdrive/MyDrive/dataset_CV_bootcamp/output/test_on_cpu"


trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()



AssertionError: Attribute 'json_file' in the metadata of 'my_train' cannot be set to a different value!
/gdrive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json != /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations.json

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file

register_coco_instances("my_val1", {}, "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json", "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images")
cfg.DATASETS.TEST = ("my_val1",)  

cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/dataset_CV_bootcamp/output/model_final.pth"


cfg.MODEL.DEVICE = "cpu"


cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  


predictor = DefaultPredictor(cfg)

[03/13 15:47:12 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/dataset_CV_bootcamp/output/model_final.pth ...


In [50]:
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultPredictor
from detectron2.evaluation import inference_on_dataset
from detectron2.data import build_detection_test_loader

from detectron2.config import get_cfg


register_coco_instances(
    "my_val2",
    {},
    "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json",
    "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images"
)


cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TEST = ("my_val2",)  
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/dataset_CV_bootcamp/output/model_final.pth"  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  

cfg.MODEL.DEVICE = "cpu"

class CustomCOCOEvaluator(COCOEvaluator):
    def __init__(self, dataset_name, output_dir=None):
        super().__init__(dataset_name, output_dir=output_dir)
        self._iou_thresholds = [0.5]  



predictor_val = DefaultPredictor(cfg)

val_loader = build_detection_test_loader(cfg, "my_val2")

evaluator = CustomCOCOEvaluator("my_val2", output_dir=cfg.OUTPUT_DIR)


results = inference_on_dataset(predictor_val.model, val_loader, evaluator)
print(results)

[03/13 12:14:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/dataset_CV_bootcamp/output/model_final.pth ...


roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


WARNING [03/13 12:14:38 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[03/13 12:14:38 d2.data.datasets.coco]: Loaded 371 images in COCO format from /content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json
[03/13 12:14:38 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|:----------:|:-------------|
|   plane    | 408          |    car     | 1240         |    ship    | 193          |
|            |              |            |              |            |              |
|   total    | 1841         |            |              |            |              |
[03/13 12:14:38 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[03/13 12:14:3

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[03/13 12:14:47 d2.evaluation.evaluator]: Inference done 1/371. Dataloading: 0.6962 s/iter. Inference: 8.8403 s/iter. Eval: 0.0002 s/iter. Total: 9.5388 s/iter. ETA=0:58:49
[03/13 12:14:57 d2.evaluation.evaluator]: Inference done 2/371. Dataloading: 0.3502 s/iter. Inference: 9.0696 s/iter. Eval: 0.0001 s/iter. Total: 9.4223 s/iter. ETA=0:57:56
[03/13 12:15:09 d2.evaluation.evaluator]: Inference done 3/371. Dataloading: 0.2350 s/iter. Inference: 10.0364 s/iter. Eval: 0.0001 s/iter. Total: 10.2734 s/iter. ETA=1:03:00
[03/13 12:15:16 d2.evaluation.evaluator]: Inference done 4/371. Dataloading: 0.1766 s/iter. Inference: 9.5083 s/iter. Eval: 0.0001 s/iter. Total: 9.6872 s/iter. ETA=0:59:15


KeyboardInterrupt: 

# Тесты на предсказание и упаковку данных в csv

In [14]:
def get_label_from_COCO(instances, image_width, image_height):
    labels = []
    for i in range(len(instances)):
        class_id = instances.pred_classes[i].item()
        bbox = instances.pred_boxes[i].tensor.cpu().numpy()[0]

        
        x_center_norm = (bbox[0] + bbox[2]) / 2 / image_width
        y_center_norm = (bbox[1] + bbox[3]) / 2 / image_height
        width_norm = (bbox[2] - bbox[0]) / image_width
        height_norm = (bbox[3] - bbox[1]) / image_height

        
        label = f"{class_id} {x_center_norm:.4f} {y_center_norm:.4f} {width_norm:.4f} {height_norm:.4f}"
        labels.append(label)  

    
    tab_separated = "\t".join(labels)  
    return f'"[{tab_separated}]"' if labels else '"[]"'

In [22]:
import os

image_path_0 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/00715b5b-2683-4d09-b1b0-7ce7f5795ade.jpg"  
image_path_1 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/0164eb7e-c10c-46b2-b5dd-1afe3dde8633.jpg"  
image_path_2 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/0015878e-6e40-4019-afa3-c0538bfce5ed.jpg"  

images = [image_path_0, image_path_1, image_path_2]

for image_path in images:
    print(image_path)
    print(f'Exist? :{os.path.exists(image_path)}')
    image = cv2.imread(image_path)
    outputs = predictor(image)

    
    v = Visualizer(image[:, :, ::-1], scale=0.8)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

    image_height, image_width = image.shape[:2]

    
    labels = get_label_from_COCO(outputs["instances"].to("cpu"), image_width, image_height)
    print(labels)

Output hidden; open in https://colab.research.google.com to view.

In [23]:
from tqdm import tqdm
import pandas as pd
import csv

image_path_0 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/00715b5b-2683-4d09-b1b0-7ce7f5795ade.jpg"  
image_path_1 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/0164eb7e-c10c-46b2-b5dd-1afe3dde8633.jpg"  
image_path_2 = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images/0015878e-6e40-4019-afa3-c0538bfce5ed.jpg"  

images = [image_path_0, image_path_1, image_path_2]
image_ids = list(map(lambda x: x.split('/')[-1], images))

test_dir = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images"


predictions = []

for image_id in tqdm(image_ids[:1]):
    image_path = os.path.join(test_dir, image_id)
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    results = predictor(image)
    instances = results['instances'].to('cpu')  

    
    formatted_label = get_label_from_COCO(instances, w, h)

    predictions.append([image_id, formatted_label])


submission_df = pd.DataFrame(predictions, columns=['id', 'label'])


submission_df.to_csv(
    '/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/submission_test.csv',
    index=False,
    quoting=csv.QUOTE_NONE,  
    escapechar='\\'  
)
print()
print('Предсказания сохранены в submission.csv')

['00715b5b-2683-4d09-b1b0-7ce7f5795ade.jpg', '0164eb7e-c10c-46b2-b5dd-1afe3dde8633.jpg', '0015878e-6e40-4019-afa3-c0538bfce5ed.jpg']


100%|██████████| 1/1 [00:12<00:00, 12.76s/it]


Предсказания сохранены в submission.csv


# Делаем предсказание

In [21]:
from detectron2.config import get_cfg
from detectron2.model_zoo import get_config_file

register_coco_instances("my_val1", {}, "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/annotations_val.json", "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/train/images")
cfg.DATASETS.TEST = ("my_val1",)  

cfg = get_cfg()
cfg.merge_from_file(get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/content/drive/MyDrive/dataset_CV_bootcamp/datasets/model_final.pth"


cfg.MODEL.DEVICE = "cpu"


cfg.DATALOADER.NUM_WORKERS = 0
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  


predictor = DefaultPredictor(cfg)

[03/13 15:48:48 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/drive/MyDrive/dataset_CV_bootcamp/datasets/model_final.pth ...


In [24]:
import pandas as pd
import os
import cv2
from tqdm import tqdm
import csv


test_dir = '/content/drive/MyDrive/dataset_CV_bootcamp/datasets/test/images'


image_ids = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]


predictions = []

for image_id in tqdm(image_ids):
    image_path = os.path.join(test_dir, image_id)
    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    results = predictor(image)
    instances = results['instances'].to('cpu')

    
    formatted_label = get_label_from_COCO(instances, w, h)

    predictions.append([image_id, formatted_label])


submission_df = pd.DataFrame(predictions, columns=['id', 'label'])


submission_df.to_csv(
    os.path.join(cfg.out),
    index=False,
    quoting=csv.QUOTE_NONE,  
    escapechar='\\'  
)
print()
print('Предсказания сохранены в submission.csv')

100%|██████████| 1068/1068 [3:04:10<00:00, 10.35s/it]


AttributeError: out

In [26]:
submission_df.to_csv(
    os.path.join("/content/drive/MyDrive/dataset_CV_bootcamp/datasets/submission.csv"),
    index=False,
    quoting=csv.QUOTE_NONE,  
    escapechar='\\'  
)